In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
import gc 
datadir = './input'

In [2]:
user_app_actived = pd.read_csv(os.path.join(datadir,'user_app_actived.csv'), header=None,names=['uId','appId'],dtype={'uId': np.str})
user_app_actived.head()

In [ ]:
user_app_actived.shape

In [ ]:
user_app_actived = pd.read_csv(os.path.join(datadir,'user_app_actived.csv'), header=None, names=['uId','appId'],
                            iterator=True, low_memory=False)
                print('start')
for i in range(30):
        try:
                temp = user_app_actived.get_chunk(100000)
                print(i)
                df1=temp.drop(columns=['appId'],axis=1)
                df1=df1.join(temp['appId'].str.split('#',expand=True).stack().reset_index(level=1,drop=True).rename('appId'))
                df1.to_csv(os.path.join('./output','uId_appId_OneByOne.csv'), mode='a',index=False,header=False)
        except StopIteration:
                print('break')
                break


In [ ]:
s=['appId',]
#删除多余行
df1=user_app_actived.drop(columns=s,axis=1)
for x in s:
    df1=df1.join(user_app_actived[x].str.split('#',expand=True).stack().reset_index(level=1,drop=True).rename(x))
df1=df1.reset_index(drop=True)

In [ ]:
#uId_appId_OneByOne
uId_appId_OneByOne = pd.read_csv(os.path.join('./output','uId_appId_OneByOne.csv'), header=None, 
                                 names=['uId','appId'],iterator=True, low_memory=False)
for i in range(1):
        try:
                temp = uId_appId_OneByOne.get_chunk(100000)
        except StopIteration:
                print('ending..........')
                break
                
temp['category_label'] = LabelEncoder().fit(temp.appId).transform(temp.appId)
temp.head()

In [ ]:
uId_appId_OneByOne = pd.read_csv(os.path.join('./output','uId_appId_OneByOne.csv')
                                 , header=None
                                 ,names=['uId','appId']
                                 ,dtype={'uId': np.str}
                                 ,index_col = 'uId'
                                )
uId_appId_OneByOne['category_label'] = LabelEncoder().fit(uId_appId_OneByOne.appId).transform(uId_appId_OneByOne.appId)
uId_appId_OneByOne.head()

In [ ]:
uId_appId_OneByOne.shape

In [ ]:
df_train = pd.read_csv(os.path.join(datadir,'age_train.csv')
                       ,header=None
                       , names=['uId','age_group']
                       ,dtype = {'uId':np.str}
                       ,index_col = 'uId')
#df_train.head()
df_test = pd.read_csv(os.path.join(datadir,'age_test.csv')
                      ,names=['uId']
                      ,dtype = {'uId':np.str}
                      ,index_col = 'uId')

In [ ]:
df_train['trainrow'] = np.arange(df_train.shape[0])
df_test['testrow'] = np.arange(df_test.shape[0])

In [ ]:
uId_app = uId_appId_OneByOne.merge(df_train,on='uId')

In [ ]:
uId_app.drop(columns=['appId'],inplace=True)

In [ ]:
uId_app.shape

In [ ]:
#特征值生成
Xtr_app = csr_matrix((np.ones(uId_app.shape[0]), (uId_app.trainrow, uId_app.category_label)), 
                      shape=(df_train.shape[0],9401))                 shape=(gatest.shape[0],napps))
Xtr_app.shape
#print('Apps data: train shape {}, test shape {}'.format(Xtr_app.shape, Xte_app.shape))

In [ ]:
Xtr_app

In [ ]:
uId_app_test = uId_appId_OneByOne.merge(df_test,on='uId')
uId_app_test.head(10)

In [ ]:
uId_app_test.drop(columns=['appId'],inplace=True)
uId_app_test.head(10)

In [ ]:
uId_app_test.isnull().sum()

In [ ]:
Xte_app = csr_matrix((np.ones(uId_app_test.shape[0]), (uId_app_test.testrow, uId_app_test.category_label)), 
                      shape=(df_test.shape[0],9401))

In [ ]:
#保存特征
from scipy import sparse
sparse.save_npz('Xtr_app9401.npz', Xtr_app)  #保存
sparse.save_npz('Xte_app9401.npz', Xte_app)  #保存

In [41]:
uId_appId_OneByOne.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
appId,101803186,9401,a00289791,2339048,NaN,NaN,NaN,NaN,NaN,NaN,NaN
category_label,1.01803e+08,NaN,NaN,NaN,4987.15,2398.25,0,3375,5215,6752,9400
